# Projeto de Redes Neurais Profundas – Treinamento
Este notebook realiza o treinamento da rede binária Wide ResNet com visualização das curvas de aprendizado.

In [ ]:
# Instalação de dependências
!pip install albumentations --quiet

In [ ]:
# Clonar o repositório com o código do projeto
!git clone --branch entrega-2 https://github.com/ICEI-PUC-Minas-PPL-CDIA/projeto-5-redes-profundas-projeto-v-produtos-industriais.git
%cd projeto-5-redes-profundas-projeto-v-produtos-industriais

In [ ]:
# Montar o Google Drive para acessar os dados
from google.colab import drive
drive.mount('/content/drive')

# Linkar os dados no diretório do projeto
!ln -s /content/drive/MyDrive/Colab\ Notebooks/DadosProdutosIndustriais data

In [ ]:
# Imports principais
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

from utils.dataloader import get_dataloaders
from bynary_wide_resnet.wrn_mcdonnell import wrn_mcdonnell

In [ ]:
# Configurações de treino
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_CLASSES = 10  # ajuste conforme necessário
EPOCHS = 20
LR = 0.01
BATCH_SIZE = 64

In [ ]:
# Inicializar modelo, dataloaders e otimizador
model = wrn_mcdonnell(depth=22, width=4, num_classes=NUM_CLASSES).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)

train_loader, test_loader = get_dataloaders(batch_size=BATCH_SIZE)

In [ ]:
# Funções auxiliares para treino e validação
def train_one_epoch(model, dataloader, criterion, optimizer):
    model.train()
    total_loss, correct, total = 0, 0, 0
    for x, y in dataloader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        correct += (out.argmax(1) == y).sum().item()
        total += y.size(0)
    return total_loss / len(dataloader), 100 * correct / total

def evaluate(model, dataloader, criterion):
    model.eval()
    total_loss, correct, total = 0, 0, 0
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            out = model(x)
            loss = criterion(out, y)
            total_loss += loss.item()
            correct += (out.argmax(1) == y).sum().item()
            total += y.size(0)
    return total_loss / len(dataloader), 100 * correct / total

In [ ]:
# Loop de treino
train_losses, test_losses = [], []
train_accs, test_accs = [], []

for epoch in range(EPOCHS):
    tloss, tacc = train_one_epoch(model, train_loader, criterion, optimizer)
    vloss, vacc = evaluate(model, test_loader, criterion)
    train_losses.append(tloss)
    test_losses.append(vloss)
    train_accs.append(tacc)
    test_accs.append(vacc)
    print(f"Época {epoch+1}: Loss={tloss:.4f}/{vloss:.4f}, Acc={tacc:.2f}%/{vacc:.2f}%")

In [ ]:
# Visualização das curvas
plt.plot(train_losses, label='Train Loss')
plt.plot(test_losses, label='Test Loss')
plt.title("Curva de Loss")
plt.xlabel("Época")
plt.ylabel("Loss")
plt.legend()
plt.show()

plt.plot(train_accs, label='Train Acc')
plt.plot(test_accs, label='Test Acc')
plt.title("Curva de Acurácia")
plt.xlabel("Época")
plt.ylabel("Acurácia (%)")
plt.legend()
plt.show()

In [ ]:
# Salvar o modelo treinado no Drive
torch.save(model.state_dict(), "/content/drive/MyDrive/wrn_mcdonnell_bnn.pth")
print("Modelo salvo com sucesso!")